In [7]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score
import os
import scipy.io
import math

from sklearn.utils import shuffle

from PIL import Image
import requests
from io import BytesIO

from tensorflow.keras.applications import resnet50
from keras.preprocessing import image

import matplotlib.pyplot as plt

import numpy as np

from keras.applications.imagenet_utils import preprocess_input, decode_predictions

import tensorflow as tf
from keras.preprocessing import image

from sklearn.model_selection import train_test_split

from scipy import spatial
from tqdm import tqdm

In [2]:
resnet50_model = resnet50.ResNet50(weights='imagenet')

102973440/102967424 [==============================] - 1s 0us/step


In [3]:
styles = pd.read_csv('/kaggle/input/fashion-product-images-dataset/fashion-dataset/styles.csv', error_bad_lines=False)

shirts = styles[styles['articleType'].isin(['Shirts'])]
tshirts = styles[styles['articleType'].isin(['Tshirts'])]
pants =  styles[styles['articleType'].isin(['Track Pants','Shorts', 'Trunk', 'Trousers', 'Track Pants', 'Tights', 'Lounge Pants', 'Lounge Shorts', 'Leggings', 'Jeans', 'Jeggings'])]
# np.unique(styles['articleType'])
shirts, tshirts, pants = shirts['id'].to_numpy(), tshirts['id'].to_numpy(), pants['id'].to_numpy()
shirts.shape, tshirts.shape, pants.shape

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

((3217,), (7067,), (2445,))

# Constants

In [5]:
image_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/'

IMG_SIZE = 224
LIMIT_IMAGES = 1500
NUM_OUTPUTS = 3

# Load Image Files

In [8]:
def load_imgs(names):
    imgs = []
    for i, image_name in enumerate(tqdm(names)):
#         if i% 50 == 0 :
#             print(f"Loading Image {i}")
        try:
            img = image.load_img(f'{image_path}{image_name}.jpg', target_size=(IMG_SIZE, IMG_SIZE))
        except:
            img = None
        if img is None:
            continue
        img = np.array(img)
        imgs.append(img)
    return np.array(imgs)

In [9]:
print("Loading Images...")
print("Shirts")
shirt_images = load_imgs(shirts[:LIMIT_IMAGES])
print("TShirts")
tshirt_images = load_imgs(tshirts[:LIMIT_IMAGES])
print("Pants")
pant_images = load_imgs(pants[:LIMIT_IMAGES])
print("Done")
shirt_images.shape, tshirt_images.shape, pant_images.shape

  0%|          | 0/1500 [00:00<?, ?it/s]

Loading Images...
Shirts


  0%|          | 0/1500 [00:00<?, ?it/s]

TShirts


  0%|          | 0/1500 [00:00<?, ?it/s]

Pants


100%|██████████| 1500/1500 [02:55<00:00,  8.54it/s]


Done


((1498, 224, 224, 3), (1500, 224, 224, 3), (1500, 224, 224, 3))

In [10]:
train_shirt_images, test_shirt_images, _, _ = train_test_split(shirt_images, np.repeat(0, shirt_images.shape[0]), test_size = 0.2)
train_shirt_images.shape, test_shirt_images.shape

((1198, 224, 224, 3), (300, 224, 224, 3))

In [11]:
train_tshirt_images, test_tshirt_images, _, _ = train_test_split(tshirt_images, np.repeat(0, tshirt_images.shape[0]), test_size = 0.2)
train_tshirt_images.shape, test_tshirt_images.shape

((1200, 224, 224, 3), (300, 224, 224, 3))

In [12]:
train_pant_images, test_pant_images, _, _ = train_test_split(pant_images, np.repeat(0, pant_images.shape[0]), test_size = 0.2)
train_pant_images.shape, test_pant_images.shape

((1200, 224, 224, 3), (300, 224, 224, 3))

In [13]:
def get_vectors(imgs):
    processed_batch = preprocess_input(imgs, mode="caffe")
    return resnet50_model.predict(processed_batch)

def get_average_vector(imgs):
    vectors = get_vectors(imgs)
    print(vectors.shape)
    return np.mean(vectors, axis=0)

def closeness(a, b):
#     print(a.shape)
#     print(b.shape)
    return 1 - spatial.distance.cosine(a, b)

def closest(vector, compared_to):
    best = -5
    best_idx = -1
#     print(compared_to.shape)
    for i, cmp in enumerate(compared_to):
        c = closeness(vector, cmp)
        if c > best:
            best_idx = i
            best = c
    return best_idx, best

def b_closest(vectors, compared_to):
    return np.array([closest(vector, compared_to)[0] for vector in vectors])

In [14]:
shirt_vector = get_average_vector(train_shirt_images)
tshirt_vector = get_average_vector(train_tshirt_images)
pant_vector = get_average_vector(train_pant_images)

(1198, 1000)
(1200, 1000)
(1200, 1000)


# Model Results

In [15]:
pred_classes = decode_predictions(np.expand_dims(shirt_vector, axis=0), top=3)
pred_classes

40960/35363 [==================================] - 0s 0us/step


[[('n03594734', 'jean', 0.47172767),
  ('n02963159', 'cardigan', 0.1194068),
  ('n03877472', 'pajama', 0.028406408)]]

In [16]:
pred_classes = decode_predictions(np.expand_dims(tshirt_vector, axis=0), top=3)
pred_classes

[[('n03595614', 'jersey', 0.4192127),
  ('n03594734', 'jean', 0.24368715),
  ('n02916936', 'bulletproof_vest', 0.029875621)]]

In [17]:
pred_classes = decode_predictions(np.expand_dims(pant_vector, axis=0), top=3)
pred_classes

[[('n03594734', 'jean', 0.44919875),
  ('n04371430', 'swimming_trunks', 0.11095806),
  ('n03255030', 'dumbbell', 0.07147162)]]

In [18]:
test_vector = np.array([shirt_vector, tshirt_vector, pant_vector])

In [19]:
train_X = np.concatenate((train_shirt_images, train_tshirt_images, train_pant_images), axis = 0)
train_Y = np.repeat((0, 1, 2), (train_shirt_images.shape[0], train_tshirt_images.shape[0], train_pant_images.shape[0]), axis = 0)

train_vecs = get_vectors(train_X)
train_predictions = b_closest(train_vecs, test_vector)

train_accuracy = accuracy_score(train_Y, train_predictions)
print(f"In Sample Accuracy: {train_accuracy}")

In Sample Accuracy: 0.6481378543635353


In [20]:
test_X = np.concatenate((test_shirt_images, test_tshirt_images, test_pant_images), axis = 0)
test_Y = np.repeat((0, 1, 2), (test_shirt_images.shape[0], test_tshirt_images.shape[0], test_pant_images.shape[0]), axis = 0)

test_vecs = get_vectors(test_X)
test_predictions = b_closest(test_vecs, test_vector)

test_accuracy = accuracy_score(test_Y, test_predictions)
print(f"Out Of Sample Accuracy: {test_accuracy}")

Out Of Sample Accuracy: 0.6711111111111111


# NN To Classify

In [21]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [22]:
trainx, trainy = shuffle(train_vecs, train_Y)
# trainx = np.expand_dims(trainx, axis=1)
trainx.shape, trainy.shape

((3598, 1000), (3598,))

In [23]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=1000, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [25]:
kfold = KFold(n_splits=10, shuffle=True)

In [26]:
classify_model = estimator.fit(trainx, trainy)

In [27]:
preds = estimator.predict(train_vecs)
acc = accuracy_score(train_Y, preds)
print(f"In sample Accuracy: {acc}")

In sample Accuracy: 0.9877709838799333


In [28]:
preds = estimator.predict(test_vecs)
acc = accuracy_score(test_Y, preds)
print(f"Out of sample Accuracy: {acc}")

Out of sample Accuracy: 0.9633333333333334


In [59]:
estimator.model.save('/kaggle/working/keras-963.h5')

# ADA to classify

In [30]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics

In [31]:
abc = AdaBoostClassifier(n_estimators=400, learning_rate=1)
abcm = abc.fit(trainx, trainy)

In [65]:
preds = abc.predict(train_vecs)
acc = accuracy_score(train_Y, preds)
print(f"In sample Accuracy: {acc}")

print(classification_report(train_Y, preds))

preds = abc.predict(test_vecs)
acc = accuracy_score(test_Y, preds)
print(f"Out of sample Accuracy: {acc}")

print(classification_report(test_Y, preds))

In sample Accuracy: 0.9769316286826014
              precision    recall  f1-score   support

           0       0.98      0.95      0.96      1198
           1       0.95      0.98      0.97      1200
           2       1.00      1.00      1.00      1200

    accuracy                           0.98      3598
   macro avg       0.98      0.98      0.98      3598
weighted avg       0.98      0.98      0.98      3598

Out of sample Accuracy: 0.9644444444444444
              precision    recall  f1-score   support

           0       0.96      0.95      0.95       300
           1       0.94      0.96      0.95       300
           2       1.00      0.99      0.99       300

    accuracy                           0.96       900
   macro avg       0.96      0.96      0.96       900
weighted avg       0.96      0.96      0.96       900



In [61]:
import pickle
pickle.dump(abc, open('/kaggle/working/ada-964.pickle', 'wb'))

# DL for classify

In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [34]:
trainx, trainy = shuffle(train_vecs, train_Y)
trainx.shape, trainy.shape
tx, vx, ty, vy = train_test_split(trainx, trainy, test_size = 0.2)
testx, testy = test_vecs.copy(), test_Y.copy()
tx.shape, ty.shape, vx.shape, vy.shape, testx.shape, testy.shape

((2878, 1000), (2878,), (720, 1000), (720,), (900, 1000), (900,))

In [35]:
scaler = MinMaxScaler()
tx = scaler.fit_transform(tx)
vx = scaler.transform(vx)
testx = scaler.transform(testx)

tx.shape, ty.shape, vx.shape, vy.shape, testx.shape, testy.shape

((2878, 1000), (2878,), (720, 1000), (720,), (900, 1000), (900,))

In [36]:
EPOCHS = 300
BATCH_SIZE = 100
LEARNING_RATE = 0.00007
NUM_FEATURES = 1000
NUM_CLASSES = NUM_OUTPUTS

In [37]:
class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [39]:
torch_model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
torch_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(torch_model.parameters(), lr=LEARNING_RATE)
print(torch_model)

MulticlassClassification(
  (layer_1): Linear(in_features=1000, out_features=512, bias=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (layer_3): Linear(in_features=128, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=3, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [40]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc) * 100
    
    return acc

In [41]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [42]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_dataset = ClassifierDataset(torch.from_numpy(tx).float(), torch.from_numpy(ty).long())
val_dataset = ClassifierDataset(torch.from_numpy(vx).float(), torch.from_numpy(vy).long())
test_dataset = ClassifierDataset(torch.from_numpy(testx).float(), torch.from_numpy(testy).long())

In [43]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE
)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [44]:
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    torch_model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = torch_model(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        torch_model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = torch_model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()
            loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

Begin training.


Epoch 001: | Train Loss: 0.92668 | Val Loss: 1.06169 | Train Acc: 75.862| Val Acc: 36.944
Epoch 002: | Train Loss: 0.68095 | Val Loss: 0.78023 | Train Acc: 100.000| Val Acc: 67.083
Epoch 003: | Train Loss: 0.55968 | Val Loss: 0.54551 | Train Acc: 100.000| Val Acc: 90.139
Epoch 004: | Train Loss: 0.48249 | Val Loss: 0.46575 | Train Acc: 100.000| Val Acc: 91.806
Epoch 005: | Train Loss: 0.41674 | Val Loss: 0.41397 | Train Acc: 100.000| Val Acc: 92.917
Epoch 006: | Train Loss: 0.36806 | Val Loss: 0.37346 | Train Acc: 100.000| Val Acc: 93.750
Epoch 007: | Train Loss: 0.32822 | Val Loss: 0.33802 | Train Acc: 100.000| Val Acc: 93.611
Epoch 008: | Train Loss: 0.29596 | Val Loss: 0.31585 | Train Acc: 100.000| Val Acc: 94.306
Epoch 009: | Train Loss: 0.26907 | Val Loss: 0.29088 | Train Acc: 100.000| Val Acc: 94.444
Epoch 010: | Train Loss: 0.24584 | Val Loss: 0.27370 | Train Acc: 100.000| Val Acc: 94.722
Epoch 011: | Train Loss: 0.22426 | Val Loss: 0.25996 | Train Acc: 100.000| Val Acc: 94.861


Epoch 092: | Train Loss: 0.01048 | Val Loss: 0.23082 | Train Acc: 100.000| Val Acc: 95.278
Epoch 093: | Train Loss: 0.01076 | Val Loss: 0.23727 | Train Acc: 100.000| Val Acc: 95.000
Epoch 094: | Train Loss: 0.01013 | Val Loss: 0.26566 | Train Acc: 100.000| Val Acc: 94.444
Epoch 095: | Train Loss: 0.01134 | Val Loss: 0.23571 | Train Acc: 100.000| Val Acc: 95.278
Epoch 096: | Train Loss: 0.01064 | Val Loss: 0.24836 | Train Acc: 100.000| Val Acc: 95.000
Epoch 097: | Train Loss: 0.01145 | Val Loss: 0.27552 | Train Acc: 100.000| Val Acc: 94.028
Epoch 098: | Train Loss: 0.01325 | Val Loss: 0.28234 | Train Acc: 100.000| Val Acc: 94.306
Epoch 099: | Train Loss: 0.01103 | Val Loss: 0.28644 | Train Acc: 100.000| Val Acc: 94.583
Epoch 100: | Train Loss: 0.01369 | Val Loss: 0.27352 | Train Acc: 100.000| Val Acc: 94.861
Epoch 101: | Train Loss: 0.01532 | Val Loss: 0.28098 | Train Acc: 100.000| Val Acc: 94.722
Epoch 102: | Train Loss: 0.01479 | Val Loss: 0.26310 | Train Acc: 100.000| Val Acc: 94.861

Epoch 183: | Train Loss: 0.00240 | Val Loss: 0.35778 | Train Acc: 100.000| Val Acc: 94.861
Epoch 184: | Train Loss: 0.00386 | Val Loss: 0.37360 | Train Acc: 100.000| Val Acc: 95.000
Epoch 185: | Train Loss: 0.00268 | Val Loss: 0.37976 | Train Acc: 100.000| Val Acc: 94.444
Epoch 186: | Train Loss: 0.00413 | Val Loss: 0.38202 | Train Acc: 100.000| Val Acc: 94.722
Epoch 187: | Train Loss: 0.00590 | Val Loss: 0.37167 | Train Acc: 100.000| Val Acc: 94.444
Epoch 188: | Train Loss: 0.00591 | Val Loss: 0.37029 | Train Acc: 100.000| Val Acc: 94.722
Epoch 189: | Train Loss: 0.00797 | Val Loss: 0.34726 | Train Acc: 100.000| Val Acc: 95.278
Epoch 190: | Train Loss: 0.00356 | Val Loss: 0.36590 | Train Acc: 100.000| Val Acc: 94.722
Epoch 191: | Train Loss: 0.00210 | Val Loss: 0.36278 | Train Acc: 100.000| Val Acc: 94.722
Epoch 192: | Train Loss: 0.00173 | Val Loss: 0.34617 | Train Acc: 100.000| Val Acc: 94.861
Epoch 193: | Train Loss: 0.00173 | Val Loss: 0.35770 | Train Acc: 100.000| Val Acc: 94.861

Epoch 274: | Train Loss: 0.00877 | Val Loss: 0.47538 | Train Acc: 100.000| Val Acc: 92.778
Epoch 275: | Train Loss: 0.01342 | Val Loss: 0.39400 | Train Acc: 100.000| Val Acc: 94.444
Epoch 276: | Train Loss: 0.00279 | Val Loss: 0.41663 | Train Acc: 100.000| Val Acc: 94.306
Epoch 277: | Train Loss: 0.00286 | Val Loss: 0.45804 | Train Acc: 100.000| Val Acc: 94.028
Epoch 278: | Train Loss: 0.00120 | Val Loss: 0.44078 | Train Acc: 100.000| Val Acc: 93.889
Epoch 279: | Train Loss: 0.00113 | Val Loss: 0.44205 | Train Acc: 100.000| Val Acc: 94.028
Epoch 280: | Train Loss: 0.00132 | Val Loss: 0.45051 | Train Acc: 100.000| Val Acc: 93.750
Epoch 281: | Train Loss: 0.00128 | Val Loss: 0.43284 | Train Acc: 100.000| Val Acc: 93.750
Epoch 282: | Train Loss: 0.00348 | Val Loss: 0.44750 | Train Acc: 100.000| Val Acc: 93.472
Epoch 283: | Train Loss: 0.00184 | Val Loss: 0.47287 | Train Acc: 100.000| Val Acc: 93.889
Epoch 284: | Train Loss: 0.00250 | Val Loss: 0.41490 | Train Acc: 100.000| Val Acc: 94.583

In [45]:
y_pred_list = []
with torch.no_grad():
    torch_model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = torch_model(X_batch)
        y_pred_softmax = torch.log_softmax(y_test_pred, dim = 1)
        _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
        y_pred_list.append(y_pred_tags.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [46]:
print(classification_report(testy, y_pred_list))

              precision    recall  f1-score   support

           0       0.94      0.97      0.96       300
           1       0.98      0.95      0.96       300
           2       0.97      0.97      0.97       300

    accuracy                           0.96       900
   macro avg       0.96      0.96      0.96       900
weighted avg       0.96      0.96      0.96       900



In [62]:
torch.save(torch_model.state_dict(), '/kaggle/working/torch-96-large.dict')

# New Model

In [48]:
trainx2, trainy2 = shuffle(train_vecs, train_Y)
trainx2.shape, trainy2.shape
tx2, vx2, ty2, vy2 = train_test_split(trainx, trainy, test_size = 0.2)
testx2, testy2 = test_vecs.copy(), test_Y.copy()
tx2.shape, ty2.shape, vx2.shape, vy2.shape, testx2.shape, testy2.shape

((2878, 1000), (2878,), (720, 1000), (720,), (900, 1000), (900,))

In [49]:
scaler = MinMaxScaler()
tx2 = scaler.fit_transform(tx2)
vx2 = scaler.transform(vx2)
testx2 = scaler.transform(testx2)

tx2.shape, ty2.shape, vx2.shape, vy2.shape, testx2.shape, testy2.shape

((2878, 1000), (2878,), (720, 1000), (720,), (900, 1000), (900,))

In [50]:
EPOCHS = 300
BATCH_SIZE = 100
LEARNING_RATE = 0.00007
NUM_FEATURES = 1000
NUM_CLASSES = NUM_OUTPUTS

In [51]:
class MulticlassClassification2(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification2, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 256)
        self.layer_4 = nn.Linear(256, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(256)
        self.batchnorm4 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_4(x)
        x = self.batchnorm4(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [52]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [53]:
torch_model2 = MulticlassClassification2(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
torch_model2.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(torch_model2.parameters(), lr=LEARNING_RATE)
print(torch_model2)



def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc) * 100
    
    return acc

accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

MulticlassClassification2(
  (layer_1): Linear(in_features=1000, out_features=512, bias=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (layer_3): Linear(in_features=128, out_features=256, bias=True)
  (layer_4): Linear(in_features=256, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=3, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [54]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_dataset2 = ClassifierDataset(torch.from_numpy(tx2).float(), torch.from_numpy(ty2).long())
val_dataset2 = ClassifierDataset(torch.from_numpy(vx2).float(), torch.from_numpy(vy2).long())
test_dataset2 = ClassifierDataset(torch.from_numpy(testx2).float(), torch.from_numpy(testy2).long())

train_loader2 = DataLoader(dataset=train_dataset2,
                          batch_size=BATCH_SIZE
)
val_loader2 = DataLoader(dataset=val_dataset2, batch_size=1)
test_loader2 = DataLoader(dataset=test_dataset2, batch_size=1)

In [55]:
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    torch_model2.train()
    for X_train_batch, y_train_batch in train_loader2:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = torch_model2(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        torch_model2.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = torch_model2(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()
            loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

Begin training.


Epoch 001: | Train Loss: 0.92620 | Val Loss: 1.07337 | Train Acc: 86.207| Val Acc: 33.889
Epoch 002: | Train Loss: 0.71906 | Val Loss: 0.79633 | Train Acc: 100.000| Val Acc: 85.833
Epoch 003: | Train Loss: 0.58070 | Val Loss: 0.53659 | Train Acc: 100.000| Val Acc: 88.750
Epoch 004: | Train Loss: 0.47279 | Val Loss: 0.43669 | Train Acc: 100.000| Val Acc: 91.250
Epoch 005: | Train Loss: 0.39915 | Val Loss: 0.36177 | Train Acc: 100.000| Val Acc: 93.056
Epoch 006: | Train Loss: 0.34064 | Val Loss: 0.31623 | Train Acc: 100.000| Val Acc: 94.167
Epoch 007: | Train Loss: 0.30224 | Val Loss: 0.27602 | Train Acc: 100.000| Val Acc: 95.278
Epoch 008: | Train Loss: 0.26775 | Val Loss: 0.25240 | Train Acc: 100.000| Val Acc: 96.111
Epoch 009: | Train Loss: 0.23665 | Val Loss: 0.22594 | Train Acc: 100.000| Val Acc: 96.528
Epoch 010: | Train Loss: 0.21386 | Val Loss: 0.20765 | Train Acc: 100.000| Val Acc: 96.528
Epoch 011: | Train Loss: 0.19387 | Val Loss: 0.18621 | Train Acc: 100.000| Val Acc: 97.083


Epoch 092: | Train Loss: 0.01252 | Val Loss: 0.05351 | Train Acc: 100.000| Val Acc: 98.333
Epoch 093: | Train Loss: 0.01038 | Val Loss: 0.05933 | Train Acc: 100.000| Val Acc: 98.333
Epoch 094: | Train Loss: 0.01058 | Val Loss: 0.05750 | Train Acc: 100.000| Val Acc: 98.472
Epoch 095: | Train Loss: 0.00949 | Val Loss: 0.05573 | Train Acc: 100.000| Val Acc: 98.611
Epoch 096: | Train Loss: 0.01163 | Val Loss: 0.05335 | Train Acc: 100.000| Val Acc: 98.333
Epoch 097: | Train Loss: 0.01161 | Val Loss: 0.05273 | Train Acc: 100.000| Val Acc: 98.056
Epoch 098: | Train Loss: 0.01372 | Val Loss: 0.06111 | Train Acc: 100.000| Val Acc: 97.917
Epoch 099: | Train Loss: 0.01174 | Val Loss: 0.06165 | Train Acc: 100.000| Val Acc: 98.333
Epoch 100: | Train Loss: 0.00787 | Val Loss: 0.07314 | Train Acc: 100.000| Val Acc: 98.056
Epoch 101: | Train Loss: 0.00903 | Val Loss: 0.06563 | Train Acc: 100.000| Val Acc: 98.194
Epoch 102: | Train Loss: 0.00961 | Val Loss: 0.06131 | Train Acc: 100.000| Val Acc: 98.194

Epoch 183: | Train Loss: 0.00416 | Val Loss: 0.07389 | Train Acc: 100.000| Val Acc: 98.194
Epoch 184: | Train Loss: 0.00365 | Val Loss: 0.09696 | Train Acc: 100.000| Val Acc: 97.778
Epoch 185: | Train Loss: 0.00252 | Val Loss: 0.09770 | Train Acc: 100.000| Val Acc: 98.194
Epoch 186: | Train Loss: 0.00439 | Val Loss: 0.08040 | Train Acc: 100.000| Val Acc: 98.056
Epoch 187: | Train Loss: 0.00589 | Val Loss: 0.07294 | Train Acc: 100.000| Val Acc: 97.778
Epoch 188: | Train Loss: 0.00609 | Val Loss: 0.07143 | Train Acc: 100.000| Val Acc: 97.917
Epoch 189: | Train Loss: 0.00796 | Val Loss: 0.08177 | Train Acc: 100.000| Val Acc: 98.056
Epoch 190: | Train Loss: 0.01320 | Val Loss: 0.07960 | Train Acc: 100.000| Val Acc: 97.500
Epoch 191: | Train Loss: 0.00565 | Val Loss: 0.08981 | Train Acc: 100.000| Val Acc: 97.639
Epoch 192: | Train Loss: 0.00524 | Val Loss: 0.07570 | Train Acc: 100.000| Val Acc: 97.778
Epoch 193: | Train Loss: 0.00721 | Val Loss: 0.08963 | Train Acc: 100.000| Val Acc: 97.778

Epoch 274: | Train Loss: 0.00233 | Val Loss: 0.07627 | Train Acc: 100.000| Val Acc: 98.194
Epoch 275: | Train Loss: 0.00283 | Val Loss: 0.09424 | Train Acc: 100.000| Val Acc: 98.194
Epoch 276: | Train Loss: 0.00274 | Val Loss: 0.07952 | Train Acc: 100.000| Val Acc: 98.194
Epoch 277: | Train Loss: 0.00187 | Val Loss: 0.09894 | Train Acc: 100.000| Val Acc: 98.056
Epoch 278: | Train Loss: 0.00241 | Val Loss: 0.11643 | Train Acc: 100.000| Val Acc: 97.917
Epoch 279: | Train Loss: 0.00247 | Val Loss: 0.08383 | Train Acc: 100.000| Val Acc: 97.778
Epoch 280: | Train Loss: 0.00157 | Val Loss: 0.07750 | Train Acc: 100.000| Val Acc: 98.472
Epoch 281: | Train Loss: 0.00125 | Val Loss: 0.08571 | Train Acc: 100.000| Val Acc: 97.917
Epoch 282: | Train Loss: 0.00233 | Val Loss: 0.10427 | Train Acc: 100.000| Val Acc: 97.639
Epoch 283: | Train Loss: 0.00124 | Val Loss: 0.09220 | Train Acc: 100.000| Val Acc: 97.778
Epoch 284: | Train Loss: 0.00139 | Val Loss: 0.10829 | Train Acc: 100.000| Val Acc: 97.778

In [56]:
y_pred_list = []
with torch.no_grad():
    torch_model2.eval()
    for X_batch, _ in test_loader2:
        X_batch = X_batch.to(device)
        y_test_pred = torch_model2(X_batch)
        y_pred_softmax = torch.log_softmax(y_test_pred, dim = 1)
        _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
        y_pred_list.append(y_pred_tags.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

print(classification_report(testy, y_pred_list))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       300
           1       0.97      0.96      0.96       300
           2       0.97      0.97      0.97       300

    accuracy                           0.96       900
   macro avg       0.96      0.96      0.96       900
weighted avg       0.96      0.96      0.96       900



In [57]:
# torch.save(torch_model2.state_dict(), '/kaggle/working/torch-2-96.dict')

In [58]:
def load_img(names):
    print(names)
    imgs = []
    for i, image_name in enumerate(names):
        if i% 50 == 0 :
            print(f"Loading Image {i}")
        img = image.load_img(f'/kaggle/input/vernacular-set/{image_name}.jpeg', target_size=(IMG_SIZE, IMG_SIZE))
        if img is None:
            continue
        img = np.array(img)
        imgs.append(img)
    return np.array(imgs)

In [ ]:
pred_imgs = load_img(list(range(1,17)))
pred_imgs.shape

In [ ]:
pred_vecs = get_vectors(pred_imgs)

In [ ]:
preds = abc.predict(pred_vecs)
map_to_names = lambda x: map(lambda y: "Shirt" if y == 0 else "T-Shirt" if y == 1 else "Pant", x)
pred_results = list(zip(map_to_names([0,1,0,0,0,0,1,0,1,1,1,0,2,2,0,0]), map_to_names(preds)))
print(pred_results)

In [ ]:
for result, image in zip(pred_results, pred_imgs):
    plt.figure()
    plt.title(f"Actual: {result[0]} | Predicted: {result[1]}")
    plt.imshow(image)
    